In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import box

### Main code

In [ ]:
# Create a GeoDataFrame for the stops dataset.
path_stops = './data_simulator/huge_dataset/dataset_simulator_trajectories.compressed.parquet.stops.parquet'
gdf_stops = pd.read_parquet(path_stops)

gdf_stops = gpd.GeoDataFrame(gdf_stops, geometry=gpd.points_from_xy(gdf_stops.lng, gdf_stops.lat), crs="EPSG:4326")
del gdf_stops['lng'], gdf_stops['lat']

# Enrich the original dataframe with some temporal information.
gdf_stops['hour_start'], gdf_stops['hour_end'] = gdf_stops['datetime'].dt.hour.astype(np.uint8), gdf_stops['leaving_datetime'].dt.hour.astype(np.uint8)
gdf_stops['weekday'] = gdf_stops['datetime'].dt.weekday.astype(np.uint8)

display(gdf_stops)
display(gdf_stops.info())

### Plot some stop segments' statistics

In [ ]:
# Show the frequency of the top-20 pairs (hour_start, hour_stop)
stop_occurrences = gdf_stops[['hour_start','hour_end']].value_counts(normalize = True)
stop_occurrences.head(30)

### Verify a user's stop segments veracity by plotting their movements

In [ ]:
user_id = 2
path_dataset = './data_simulator/huge_dataset/dataset_simulator_trajectories.parquet'

# Read the parquet file containing the simulator's trajectories.
# NOTE: we are selecting only the rows that satisfy the conditions in 'sel' to save memory!
sel = [("ID", "in", [user_id])]
users = pd.read_parquet(path_dataset, filters = sel)
users = gpd.GeoDataFrame(users, geometry=gpd.points_from_xy(users.lon, users.lat), crs="EPSG:4326")

display(users.info())
display(users)

In [ ]:
# Show the stop segments associated with the user of interest.
gdf_stops.loc[gdf_stops['uid'] == user_id]

In [ ]:
# Plot how the users move over time.
import folium
from folium.plugins import TimestampedGeoJson

# 2. Pick a palette of colors and map each unique ID to one
palette = [
    "red", "blue", "green", "purple", "orange",
    "darkred", "lightred", "beige", "darkblue",
    "darkgreen", "cadetblue", "darkpurple"
]
unique_ids = users["ID"].unique()
color_map = {
    uid: palette[i % len(palette)]
    for i, uid in enumerate(unique_ids)
}


# 2. Build a GeoJSON-style dict with times
features = []
for _, row in users.iterrows():
    cid = row["ID"]
    features.append({
        "type": "Feature",
        "geometry": row.geometry.__geo_interface__,
        "properties": {
            "time": row.timestamp.isoformat(),
            "icon": "circle",
            "iconstyle": {
                "fillColor": color_map[cid],
                "fillOpacity": 0.7,
                "stroke": False,
                "radius": 10
            },
            "popup": f"ID {cid}<br>{row.timestamp:%Y-%m-%d %H:%M}"
        }
    })

geojson = {"type": "FeatureCollection", "features": features}


# 3. Create the map
m = folium.Map(location=[users.geometry.y.mean(), users.geometry.x.mean()], zoom_start=13, control_scale=True)

# 4. Add the time‐slider layer
duration_step_minutes = (users.iloc[1]['timestamp'] - users.iloc[0]['timestamp']).seconds // 60
TimestampedGeoJson(
    data=geojson,
    transition_time=200,      # milliseconds between frames
    period=f"PT{duration_step_minutes}M",            # matches your n-minutes stepping
    duration=f"PT{duration_step_minutes}M",
    add_last_point=False,
    auto_play=False,
    loop=False,
).add_to(m)


# 5. Display or save
m
# m.save("trajectory_time_slider.html")